In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [14]:
#testimage=r'G:\face-mask-detector\examples\example_03.png'
testimage=r'G:\python\aushitest2.jpg'
path1=r'G:\face-mask-detector\face_detector\deploy.prototxt'
path2=r'G:\face-mask-detector\face_detector\res10_300x300_ssd_iter_140000.caffemodel'

In [3]:
net=cv2.dnn.readNet(path1,path2)

In [4]:
model=load_model('face_mask1')

In [15]:
image=cv2.imread(testimage)
orig=image.copy()
(h,w)=image.shape[:2]

In [16]:
blob=cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123))

In [17]:
net.setInput(blob)
detections=net.forward()

In [18]:
for i in range(0,detections.shape[2]):
    confid=detections[0,0,i,2]
    
    if confid>.5:
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startx,starty,endx,endy)=box.astype('int')
        
        (startx1,starty1)=(max(0,startx),max(0,starty))
        (endx1,endy1)=(min(w-1,endx),min(h-1,endy))

In [19]:
face=image[starty1:endy1,startx1:endx1]
face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
face=cv2.resize(face,(224,224))
face=img_to_array(face)
face=preprocess_input(face)
face=np.expand_dims(face,axis=0)

In [20]:
(mask,withoutmask)=model.predict(face)[0]

In [21]:
label='Mask' if mask>withoutmask else 'No Mask'
color=(0,255,0) if label=='Mask' else (0, 0,255)
label="{}:(:.2f)%".format(label,max(mask,withoutmask)*100)

In [22]:
cv2.putText(image,label,(startx1,starty1-10),
           cv2.FONT_HERSHEY_SIMPLEX,.45,color,2
           )
cv2.rectangle(image,(startx1,starty1),(endx1,endy1),color,2)

array([[[ 72,  80,  80],
        [ 72,  80,  80],
        [ 70,  80,  80],
        ...,
        [ 81, 100,  97],
        [ 81, 100,  97],
        [ 81, 100,  97]],

       [[ 72,  80,  80],
        [ 72,  80,  80],
        [ 70,  80,  80],
        ...,
        [ 81, 100,  97],
        [ 81, 100,  97],
        [ 81, 100,  97]],

       [[ 72,  80,  80],
        [ 72,  80,  80],
        [ 70,  80,  80],
        ...,
        [ 81, 100,  97],
        [ 81, 100,  97],
        [ 81, 100,  97]],

       ...,

       [[ 52,  55,  53],
        [ 52,  55,  53],
        [ 52,  55,  53],
        ...,
        [143, 177, 190],
        [143, 177, 190],
        [143, 177, 190]],

       [[ 52,  55,  53],
        [ 52,  55,  53],
        [ 52,  55,  53],
        ...,
        [145, 177, 190],
        [145, 177, 190],
        [145, 177, 190]],

       [[ 52,  55,  53],
        [ 52,  55,  53],
        [ 52,  55,  53],
        ...,
        [145, 177, 190],
        [145, 177, 190],
        [145, 177, 190]]

In [23]:
cv2.imshow('Output',image)
cv2.waitKey(0)
cv2.destroyAllWindows()